<a href="https://colab.research.google.com/github/IkramuddinAhmed/ADL_Project/blob/main/forecast_v2_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("/content/natural_calamities-4.xls")
df

,Date,Year,State,State_Label,Calamity
0,5/2/53,1953,GA,12,Tornado
1,5/15/53,1953,TX,49,Tornado
2,5/29/53,1953,LA,21,Flood
3,6/2/53,1953,MI,26,Tornado
4,6/6/53,1953,MT,31,Flood
...,...,...,...,...,...
24826,12/10/21,2022,TN,48,Tornado
24827,12/10/21,2022,TN,48,Tornado
24828,12/10/21,2022,TN,48,Tornado
24829,12/10/21,2022,TN,48,Tornado


In [ ]:
# Replace '/' with '-'
df["Date"] = df["Date"].apply(lambda x: x.replace("/", "-"))

# Sort df by field "year" in ascending
df.sort_values(by="Year", inplace=True, ascending=True, ignore_index=True)

# Remove row whose Date value is 1989-01-01T00:00:00Z and others
df = df[df["Date"] != "1989-01-01T00:00:00Z"]
df = df[df["Date"] != "2001-03-23T08:00:00Z"]
df = df[df["Date"] != "2001-03-01T14:00:00Z"]
df = df[df["Date"] != "2001-03-01T00:00:00Z"]
df = df[df["Date"] != "2001-02-28T08:54:00Z"]
df = df[df["Date"] != "2001-04-18T13:05:00Z"]
df = df[df["Date"] != "2001-04-10T14:00:00Z"]
df = df[df["Date"] != "2001-10-13T14:15:00Z"]

# This function is to convert two digit year to full four difit year
def format_date(date):
    split = date.split("-")
    year = int(split[-1])
    if 40 <= year <= 99:
        date = split[0] + "-" + split[1] + "-" + "19" + split[2]
    else:
        date = split[0] + "-" + split[1] + "-" + "20" + split[2]
    return date

df["Date"] = df["Date"].apply(lambda x: format_date(x))
df.drop_duplicates(subset=["Date", "State", "Calamity"], keep="first", inplace=True) # drop duplicates

df.reset_index(drop=True, inplace=True)

In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%Y")    # Convert to datetime
df.sort_values(by="Date", inplace=True, ascending=True, ignore_index=True) # Sort by Date in ascending

In [ ]:
df

,Date,Year,State,State_Label,Calamity
0,1953-05-02,1953,GA,12,Tornado
1,1953-05-15,1953,TX,49,Tornado
2,1953-05-29,1953,LA,21,Flood
3,1953-06-02,1953,MI,26,Tornado
4,1953-06-06,1953,MT,31,Flood
...,...,...,...,...,...
1397,2021-11-13,2022,WA,54,Flood
1398,2021-12-10,2022,KY,20,Tornado
1399,2021-12-10,2022,TN,48,Tornado
1400,2021-12-10,2022,IL,17,Tornado


# For Texas state

In [ ]:
tx_df = df[df["State"] == "TX"].copy()
tx_df.drop(columns=["Year", "State", "State_Label"], inplace=True) # Drop unwanted columns
tx_df.reset_index(drop=True, inplace=True)
tx_df

,Date,Calamity
0,1953-05-15,Tornado
1,1953-06-19,Flood
2,1954-07-01,Flood
3,1957-04-29,Hurricane
4,1958-06-06,Tornado
...,...,...
72,2018-09-10,Flood
73,2019-06-24,Flood
74,2019-09-17,Flood
75,2020-07-25,Hurricane


In [ ]:
tx_df.dtypes

Date        datetime64[ns]
Calamity            object
dtype: object

In [ ]:
tx_df.iloc[0]["Date"], tx_df.iloc[-1]["Date"]

(Timestamp('1953-05-15 00:00:00'), Timestamp('2020-08-23 00:00:00'))

## Create bins of 3 months for column Dateand the bin range will be mentioned in "bins" columns

In [ ]:
from random import randrange
from datetime import timedelta, date
import numpy as np
import pandas as pd


start = tx_df.iloc[0]["Date"] # Start date
end = tx_df.iloc[-1]["Date"]  # end date
dftx = tx_df[["Date"]].copy()  
bins = pd.date_range(start, end, freq='3MS')  # Create bins of 3 Months

dftx["bins"] = pd.cut(dftx.Date, bins=bins)  # Get bins for each Date 
dftx.sort_values(by="bins", inplace=True, ascending=True, ignore_index=True) # Sort by bins column
dftx

,Date,bins
0,1953-06-19,"(1953-06-01, 1953-09-01]"
1,1954-07-01,"(1954-06-01, 1954-09-01]"
2,1957-04-29,"(1957-03-01, 1957-06-01]"
3,1958-06-06,"(1958-06-01, 1958-09-01]"
4,1959-07-08,"(1959-06-01, 1959-09-01]"
...,...,...
72,2019-06-24,"(2019-06-01, 2019-09-01]"
73,2019-09-17,"(2019-09-01, 2019-12-01]"
74,1953-05-15,NaN
75,2020-07-25,NaN


In [ ]:
# Join the two dataframes tx_df and dftx on Date column
# tx_df has Date, Calamity while dftx has Date and bins.
# We will join them together to get Date, calamity and its bin
dfv = pd.merge(tx_df, dftx, on="Date", how="left")
dfv

,Date,Calamity,bins
0,1953-05-15,Tornado,NaN
1,1953-06-19,Flood,"(1953-06-01, 1953-09-01]"
2,1954-07-01,Flood,"(1954-06-01, 1954-09-01]"
3,1957-04-29,Hurricane,"(1957-03-01, 1957-06-01]"
4,1958-06-06,Tornado,"(1958-06-01, 1958-09-01]"
...,...,...,...
72,2018-09-10,Flood,"(2018-09-01, 2018-12-01]"
73,2019-06-24,Flood,"(2019-06-01, 2019-09-01]"
74,2019-09-17,Flood,"(2019-09-01, 2019-12-01]"
75,2020-07-25,Hurricane,NaN


In [ ]:
dfv["bins"] = dfv["bins"].apply(lambda x: x.left) # Select the starting date of bins, i.e. this will be indicator start of 3 months or a quater
dfv["bins"] = pd.to_datetime(dfv["bins"])         # Convert to datetime

In [ ]:
# Create a separate bins df
temp_df = pd.DataFrame({"bins" : bins.tolist()})    
temp_df["bins"] = pd.to_datetime(temp_df["bins"])

In [ ]:
dfv

,Date,Calamity,bins
0,1953-05-15,Tornado,NaT
1,1953-06-19,Flood,1953-06-01
2,1954-07-01,Flood,1954-06-01
3,1957-04-29,Hurricane,1957-03-01
4,1958-06-06,Tornado,1958-06-01
...,...,...,...
72,2018-09-10,Flood,2018-09-01
73,2019-06-24,Flood,2019-06-01
74,2019-09-17,Flood,2019-09-01
75,2020-07-25,Hurricane,NaT


In [ ]:
dfv.dtypes

Date        datetime64[ns]
Calamity            object
bins        datetime64[ns]
dtype: object

In [ ]:
temp_df.dtypes

bins    datetime64[ns]
dtype: object

In [ ]:
# join dfv , temp_df on bins column 
# So, we have data for every 3 months, wherever we dont have data Calamity is Nan there.
# We will replace Nan with "No_calamity" as no clalmity happened in this 3 months
dfv = pd.merge(dfv, temp_df, on="bins", how="right")

In [ ]:
dfv

,Date,Calamity,bins
0,1953-06-19,Flood,1953-06-01
1,NaT,NaN,1953-09-01
2,NaT,NaN,1953-12-01
3,NaT,NaN,1954-03-01
4,1954-07-01,Flood,1954-06-01
...,...,...,...
276,2019-06-24,Flood,2019-06-01
277,2019-09-17,Flood,2019-09-01
278,NaT,NaN,2019-12-01
279,NaT,NaN,2020-03-01


In [ ]:
dfv.Calamity = dfv.Calamity.fillna('No_Calamity') # Fill Nan with "No_Calamity"

In [ ]:
dfv

,Date,Calamity,bins
0,1953-06-19,Flood,1953-06-01
1,NaT,No_Calamity,1953-09-01
2,NaT,No_Calamity,1953-12-01
3,NaT,No_Calamity,1954-03-01
4,1954-07-01,Flood,1954-06-01
...,...,...,...
276,2019-06-24,Flood,2019-06-01
277,2019-09-17,Flood,2019-09-01
278,NaT,No_Calamity,2019-12-01
279,NaT,No_Calamity,2020-03-01


In [ ]:
dfv["Calamity"].value_counts()    # The count of calamities for texas

No_Calamity    207
Flood           40
Hurricane       20
Tornado         14
Name: Calamity, dtype: int64

In [ ]:
# Convert String Calamities to numeric
# The mapping from numeric to its original calamity string is shown in calamity_to_numeric_dict
calamity_to_numeric_dict = dict(enumerate(dfv.Calamity.astype("category").cat.categories))
print(calamity_to_numeric_dict)

{0: 'Flood', 1: 'Hurricane', 2: 'No_Calamity', 3: 'Tornado'}


In [ ]:
# Convert string calamity to numeric counterpart.
# i.e. {0: 'Flood', 1: 'Hurricane', 2: 'No_Calamity', 3: 'Tornado'}
dfv["Calamity"] = dfv.Calamity.astype("category").cat.codes

In [ ]:
dfv["Calamity"].value_counts() # Count of calamity in numeric form

2    207
0     40
1     20
3     14
Name: Calamity, dtype: int64

In [ ]:
#data  = dfv["Calamity"].tolist()

In [ ]:
events = list(dfv['Calamity'].unique())  # total Unique count of calamities. Here it is 4

In [ ]:
# LSTM needs sequence of data.So, we will take sequence of 13 numeric calamities to predict the next calamity
series_length = 13        
categories = len(events)      # Total different types of calamities
time_steps = series_length -1 # Timestep will be 13 -1 = 12

In [ ]:
categories, time_steps

(4, 12)

In [ ]:
# Create list of list having 12 sequences
history = []
for x in range(len(dfv['Calamity'].values)-series_length):
    history.append(dfv['Calamity'].values[x:x+series_length])

In [ ]:
X= np.array([row[:time_steps] for row in history])

In [ ]:
X

array([[0, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 0, ..., 2, 2, 2],
       ...,
       [2, 2, 2, ..., 2, 2, 0],
       [2, 2, 1, ..., 2, 0, 0],
       [2, 1, 2, ..., 0, 0, 2]], dtype=int8)

In [ ]:
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.metrics import top_k_categorical_accuracy

# Keras expects dat in categorical format.
X = to_categorical(X)

In [ ]:
# similarly convert y to categorical
y=np.array([row[time_steps] for row in history])
y = to_categorical(y).reshape(-1,categories)

In [ ]:
X.shape, y.shape

((268, 12, 4), (268, 4))

In [ ]:
# Since this timeseries data, we will take first 95% of data as train and rest 15% data as test

upperlimit = round(len(X)*0.95)

X_train, X_test = X[:upperlimit,:], X[upperlimit:,:]
y_train, y_test = y[:upperlimit,:], y[upperlimit:,:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(255, 12, 4) (13, 12, 4) (255, 4) (13, 4)


In [ ]:
# Function to get top1, to2 and top3 accuracy
# Top-1 accuracy is the conventional accuracy, model prediction (the one with the highest probability) must be exactly the expected answer.

# Top-2 accuracy means any of our model’s top 2 highest probability answers match with the expected answer.
# It considers a classification correct if any of the two predictions matches the target label.

# Top-3 accuracy means any of our model’s top 3 highest probability answers match with the expected answer.
# It considers a classification correct if any of the three predictions matches the target label.

def top3acc(Y_true, Y_pred):
  return top_k_categorical_accuracy(Y_true, 
                                    Y_pred, 
                                    k = 3)

def top2acc(Y_true, Y_pred):
  return top_k_categorical_accuracy(Y_true, 
                                    Y_pred, 
                                    k = 2)
def top1acc(Y_true, Y_pred):
  return top_k_categorical_accuracy(Y_true, 
                                    Y_pred, 
                                    k = 1)


metrics = [top3acc, top2acc, top1acc] # List of metrics

model = Sequential() 

model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(time_steps,categories)))
model.add(Dropout(0.2))

#model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(time_steps,128)))
#model.add(Dropout(0.4))

model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(categories, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 12, 64)            17664     
                                                                 
 dropout_2 (Dropout)         (None, 12, 64)            0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 50,948
Trainable params: 50,948
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train, epochs=200) # fit train data

Epoch 1/200
8/8 [==============================] - 2s 13ms/step - loss: 1.3155 - top3acc: 0.8980 - top2acc: 0.7961 - top1acc: 0.6431
Epoch 2/200
8/8 [==============================] - 0s 13ms/step - loss: 1.1137 - top3acc: 0.9059 - top2acc: 0.8118 - top1acc: 0.7373
Epoch 3/200
8/8 [==============================] - 0s 16ms/step - loss: 0.9128 - top3acc: 0.8902 - top2acc: 0.8235 - top1acc: 0.7373
Epoch 4/200
8/8 [==============================] - 0s 13ms/step - loss: 0.8604 - top3acc: 0.9373 - top2acc: 0.8549 - top1acc: 0.7373
Epoch 5/200
8/8 [==============================] - 0s 12ms/step - loss: 0.8546 - top3acc: 0.9373 - top2acc: 0.8784 - top1acc: 0.7373
Epoch 6/200
8/8 [==============================] - 0s 12ms/step - loss: 0.8583 - top3acc: 0.9451 - top2acc: 0.8627 - top1acc: 0.7373
Epoch 7/200
8/8 [==============================] - 0s 14ms/step - loss: 0.8588 - top3acc: 0.9451 - top2acc: 0.8667 - top1acc: 0.7373
Epoch 8/200
8/8 [==============================] - 0s 13ms/step - los

# As you can see top1acc 80.39% , top2 accuracy 92.94%, top3 accuracy 98.43%

In [ ]:
# Predict for test data
prediction = model.predict(X_test)
np.argmax(prediction, axis=1)

array([2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2])

In [ ]:
# actaul y_test
np.argmax(y_test, axis=1)

array([2, 1, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2])

In [ ]:
from sklearn.metrics import classification_report
y_true = list(np.argmax(y_test, axis=1))
y_pred = list(np.argmax(prediction, axis=1))
target_names = ['Flood', 'Hurricane', 'No_Calamity', 'Tornado']

In [ ]:
# Classification report for all calamities

In [ ]:
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Flood       0.00      0.00      0.00         4
   Hurricane       0.00      0.00      0.00         1
 No_Calamity       0.64      0.88      0.74         8
     Tornado       0.00      0.00      0.00         0

    accuracy                           0.54        13
   macro avg       0.16      0.22      0.18        13
weighted avg       0.39      0.54      0.45        13



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1